# Data Mining Project
### Giorgio Donati, g.donati24@studenti.unipi.it
### Pietro Francaviglia, p.francaviglia1@studenti.unipi.it
#### A.Y. 2021-2022


## Libraries and datasets

In [1]:
from sklearn.ensemble import RandomForestClassifier

from t3_constants import *
from t3_utility import *

# Random Forest
## Training

In [2]:
with open(f'{PICKLE_FOLDER}{TRAIN}{X}.pkl', 'rb') as f:
    train_x = pickle.load(f)
with open(f'{PICKLE_FOLDER}{TRAIN}{Y}.pkl', 'rb') as f:
    train_y = pickle.load(f)

x = train_x.values
y = train_y.values.ravel().astype(str)

In [3]:
# Perform the hyper-parameters grid-search on random forest
param_d = dict(
    n_estimators = np.logspace(1, 2.5, num=10).astype(int).tolist(),
    max_features = ['sqrt'],
    max_depth = [None] + np.logspace(0.5, 2, num=9).astype(int).tolist(),
    min_samples_split = [2, 5, 10],
    min_samples_leaf = [1, 2, 4],
    bootstrap = [True, False]
)

best_params = randomized_cv(
    model=RandomForestClassifier(random_state=0),
    x=x,
    y=y,
    param_d=param_d,
    n_iter=1000
)

RandomForestClassifier(max_depth=17, n_estimators=215, random_state=0)
 mean acc: 0.890
 std_acc: 0.026


## Testing

In [4]:
with open(f'{PICKLE_FOLDER}{TEST}{X}.pkl', 'rb') as f:
    test_x = pickle.load(f)
with open(f'{PICKLE_FOLDER}{TEST}{Y}.pkl', 'rb') as f:
    test_y = pickle.load(f)


In [7]:
best_params.score(test_x.values, test_y.values.ravel())

0.8662420382165605

In [6]:
# model = RandomForestClassifier()